We illustrate how we might write a class that indexes like numpy arrays and torch tensors.

In [1]:
from fastai.vision.all import *

pickle_path = URLs.path('mnist_png')/'mnist_png.pkl'
path = untar_data(URLs.MNIST)/'training'

if not pickle_path.exists():
    pickle_path.parent.mkdir(parents=True, exist_ok=True)
    ds = DataBlock(
        blocks = (ImageBlock(PILImageBW), CategoryBlock),
        get_items = get_image_files,
        get_y = parent_label,
        splitter = RandomSplitter(1/6, seed=0)
    ).datasets(path)

    xs, ys = zip(*ds.train, *ds.valid)
    xs = np.stack(L(map(lambda x: np.array(x, dtype=np.float32).reshape(-1), xs))) / 255.
    ys = np.array(ys, dtype=np.int64)

    x_train, x_valid = xs[:len(ds.train)], xs[len(ds.train):]
    y_train, y_valid = ys[:len(ds.train)], ys[len(ds.train):]

    save_pickle(pickle_path, [x_train, y_train, x_valid, y_valid])

    del ds, xs, ys, x_train, y_train, x_valid, y_valid

import torch
from torch import tensor

x_train, y_train, x_valid, y_valid = map(tensor, load_pickle(pickle_path))
x_train.shape, x_train.type()


(torch.Size([50000, 784]), 'torch.FloatTensor')

In [2]:
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

(torch.Size([50000, 784]),
 torch.Size([10000, 784]),
 torch.Size([50000]),
 torch.Size([10000]))

We create a model that performs the following:

`dataset` (N-by-784) @ `weights` (784-by-10) + `bias` (10) = `preds` (N-by-10),

where `@` indicated matrix multiplication, we have abstracted away the activation functions, and the 10 output dimensions encode a probability distribution for the model's convidence in the input being each digit.

In [3]:
torch.manual_seed(1)
weights = torch.randn(784, 10)
bias = torch.zeros(10)


Demo of matmul as dotproduct on a slice.

In [4]:
m1 = x_valid[:5]
m2 = weights

ar, ac = m1.shape
br, bc = m2.shape
(ar, ac), (br, bc)

t1 = torch.zeros(ar, bc)
print(t1.shape)

for i in range(ar):         # 5
    for j in range(bc):     # 10
        for k in range(ac): # 784
            t1[i, j] += m1[i, k] * m2[k, j]

t1

torch.Size([5, 10])


tensor([[ -7.8318,  -9.3646,   6.0571, -14.2204,  20.7075,   1.9792, -20.8511,
         -14.9247, -24.9726,  -1.5234],
        [  9.5953,   8.4150,   8.4275,  -7.7321,  11.0846, -26.4744,  -3.6933,
          -9.4670, -35.0040,   7.3345],
        [  4.7560,  -3.3547,  -8.8216,  11.5083,  -5.3496,  -2.2799, -14.0786,
           1.3523,  -7.9863,  -3.3055],
        [  6.8275,   1.2459,  -0.5831,  13.8737,  -1.6997,  -7.8282,  -1.4413,
           0.4070, -17.4009,  -7.7779],
        [-11.6686, -12.9786,  -7.4251,  -4.4291,  -1.6932,   3.1152,  -5.8956,
          -5.2578, -15.3839,   3.5509]])

In [5]:
t1.shape


torch.Size([5, 10])

We abstract it to its own function.

In [6]:
def matmul(a, b):
    (ar, ac), (br, bc) = a.shape, b.shape
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            for k in range(ac):
                c[i, j] += a[i, k] * b[k, j]
    return c


And time how long it takes to run.

In [7]:
%time _ = matmul(m1, m2)


CPU times: user 431 ms, sys: 0 ns, total: 431 ms
Wall time: 430 ms


The innermost loop of matmul is run the following number of times

In [8]:
ar * bc * ac


39200